# GitHub Gist Documentation Uploader

This notebook will help you upload your Markdown documentation to a GitHub Gist and provide a URL for use in Cursor. It can handle both local directories and GitHub repository URLs.

In [27]:
import sys
import subprocess
import os
import requests
from github import Github, InputFileContent
from dotenv import load_dotenv
from IPython.display import display, HTML
import re

# Function to install required packages
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install required packages
required_packages = ["requests", "PyGithub", "python-dotenv", "ipython"]
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        install_package(package)
        print(f"{package} installed successfully.")

PyGithub not found. Installing...
PyGithub installed successfully.
python-dotenv not found. Installing...
python-dotenv installed successfully.
ipython not found. Installing...
ipython installed successfully.


## Setup
First, let's check if we have a GitHub token stored in a .env file. If not, we'll prompt for one.

In [28]:
load_dotenv()
token = os.getenv("GITHUB_TOKEN")

if not token:
    print("To create a GitHub Personal Access Token, please follow these instructions:")
    print("1. Go to https://github.com/settings/tokens")
    print("2. Click 'Generate new token' (classic)")
    print("3. Give your token a descriptive name")
    print("4. Set the following permissions:")
    print("   - In the 'Repository permissions' section:")
    print("     * Contents: Read-only")
    print("     * Metadata: Read-only")
    print("   - In the 'Account permissions' section:")
    print("     * Gists: Read and write")
    print("5. Click 'Generate token' at the bottom of the page")
    print("6. Copy the created token and paste it below")
    print("\nWARNING: The token will only be shown once. Save it in a secure location.")

    token = input("\nPlease enter your GitHub Personal Access Token: ")

    # Save token to .env file
    with open(".env", "w") as env_file:
        env_file.write(f"GITHUB_TOKEN={token}")

    print("Token saved to .env file.")
else:
    print("Token loaded from .env file.")

Token loaded from .env file.


## Set Documentation Source and Gist Name
Now, let's set the path or URL to your documentation and name for the Gist.

In [29]:
docs_source = input("Enter the path to your documentation directory or GitHub URL: ")
gist_name = input("Enter a name for your Gist: ")

## Define Helper Functions

In [30]:
def is_github_url(url):
    return url.startswith("https://github.com/")

def get_github_contents(repo_url):
    # Extract owner, repo, branch, and path from the URL
    match = re.match(r'https://github.com/([^/]+)/([^/]+)/tree/([^/]+)/(.+)', repo_url)
    if not match:
        raise ValueError("Invalid GitHub URL")

    owner, repo, branch, path = match.groups()

    def get_contents(path):
        api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={branch}"
        response = requests.get(api_url, headers={"Authorization": f"token {token}"})
        response.raise_for_status()
        return response.json()

    def process_contents(contents, current_path=""):
        md_files = {}
        for item in contents:
            if item['type'] == 'file' and item['name'].endswith('.md'):
                file_content = requests.get(item['download_url']).text
                md_files[os.path.join(current_path, item['name'])] = file_content
            elif item['type'] == 'dir':
                subdir_contents = get_contents(item['path'])
                md_files.update(process_contents(subdir_contents, os.path.join(current_path, item['name'])))
        return md_files

    initial_contents = get_contents(path)
    return process_contents(initial_contents)

def get_local_md_files(directory):
    md_files = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    md_files[file] = f.read()
    return md_files

## Upload Files to Gist

In [31]:
# Create a Github instance
g = Github(token)

# Get all MD files
if is_github_url(docs_source):
    md_files = get_github_contents(docs_source)
else:
    md_files = get_local_md_files(docs_source)

# Create a new gist
gist_files = {}
for file_name, content in md_files.items():
    gist_files[file_name] = InputFileContent(content)

# Get user's gists
user = g.get_user()
gists = user.get_gists()

# Try to find existing gist with the same name
existing_gist = None
for gist in gists:
    if gist.description == gist_name:
        existing_gist = gist
        break

if existing_gist:
    print(f"A Gist with the name '{gist_name}' already exists.")
    choice = input("Do you want to update the existing Gist or create a new one? (update/new): ").lower()

    if choice == 'update':
        # Update existing gist
        existing_gist.edit(description=gist_name, files=gist_files)
        gist = existing_gist
        print("Existing Gist updated.")
    else:
        # Create new gist
        gist = g.get_user().create_gist(public=True, files=gist_files, description=gist_name)
        print("New Gist created.")
else:
    # Create new gist
    gist = g.get_user().create_gist(public=True, files=gist_files, description=gist_name)
    print("New Gist created.")

# Display the gist URL
display(HTML(f'<p>Gist URL: <a href="{gist.html_url}" target="_blank">{gist.html_url}</a></p>'))
print("Use this URL in Cursor to add your documentation.")

A Gist with the name 'Godot XR-Tools docs for llm ' already exists.
Existing Gist updated.


Use this URL in Cursor to add your documentation.
